In [1]:
#!pip install torch==1.13.0+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install transformers==4.30.0
!pip install datasets==2.13.2
!pip install accelerate -U
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.1
    Uninstalling tokenizers-0.15.1:
      Successfully uninstalled tokenizers-0.15.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━

In [1]:
# system packages
from pathlib import Path
import shutil
import urllib
import tarfile
import sys
import os
# data and numerical management packages
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, AdamW, Trainer, TrainingArguments
from torch.nn.utils.rnn import pad_sequence

import warnings
warnings.filterwarnings('ignore')
# useful during debugging (progress bars)
from tqdm import tqdm
from transformers import set_seed

seed = 852

random.seed(seed)
np.random.seed(seed)
set_seed(seed)

In [6]:
ls

sample_data/


In [7]:
dataset_folder = Path.cwd().joinpath('sample_data/MELD_train_efr.json')
#dataset_path = dataset_folder.joinpath('/MELD_train_efr.json')
#dataset_folder = "/kaggle/input/plaplapla/MELD_train_efr.json"
df = pd.read_json(dataset_folder)
#df['triggers'] = df['triggers'].fillna(value=0, inplace=False)#.replace('None', 0.0)

In [8]:
df

,episode,speakers,emotions,utterances,triggers
0,utterance_0,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise]",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 1.0, 0.0]"
1,utterance_1,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
2,utterance_2,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
3,utterance_3,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,utterance_4,"[Joey, Rachel, Joey, Rachel]","[surprise, sadness, surprise, fear]",[But then who? The waitress I went out with la...,"[0.0, 0.0, 1.0, 0.0]"
...,...,...,...,...,...
3995,utterance_3995,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3996,utterance_3996,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3997,utterance_3997,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3998,utterance_3998,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [9]:
triggers = df['triggers']
for row in range(triggers.shape[0]):
    for trigger in range(len(triggers[row])):
        if triggers[row][trigger] == None:
            triggers[row][trigger] = 0.0

df['triggers'] = triggers

In [10]:
emotions = df['emotions'].explode().unique()
emotions

array(['neutral', 'surprise', 'fear', 'sadness', 'joy', 'disgust',
       'anger'], dtype=object)

In [11]:
triggers = df['triggers'].explode().unique()
triggers

array([0.0, 1.0], dtype=object)

In [12]:
dialogues = df['utterances'][:3200]
#print(sentences)
max_len_utterance = 0
index = 0
utterances_len = []
for dialogue in dialogues:
  for utterance in dialogue:
    #print(utterance)
    #if len(utterance.split()) > max_len_utterance:
     # max_len_utterance = len(utterance.split())
     utterances_len.append(len(utterance.split()))
np.mean(np.array(utterances_len))

8.077553661956639

In [13]:
from sklearn.preprocessing import LabelBinarizer
sorted_emotions = sorted(emotions)  #sort the array because Binarizer will automatically do that for one hot encoding
label_binarizer = LabelBinarizer()
label_binarizer.fit(sorted_emotions)

dialogues = df['emotions']
one_hot_emotions = []
for dialogue_emotion in dialogues:
  dialogue_emotions_list = []
  for emotion in dialogue_emotion:
    encoded_emotion=label_binarizer.transform([emotion])
    dialogue_emotions_list.append(np.ravel(encoded_emotion).tolist())
  one_hot_emotions.append(dialogue_emotions_list)

In [14]:
df['emotions'] = one_hot_emotions

In [15]:
from sklearn.model_selection import train_test_split
train_data, temp_data = train_test_split(df, train_size=0.8, shuffle=False)
val_data, test_data = train_test_split(temp_data, test_size=0.5, shuffle=False)
val_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_padding(dialogues):
  tokenized_dialogues = []
  max_length = 10

  for dialogue in tqdm(dialogues):
      tokenized_dialogue = tokenizer(
          dialogue,
          max_length=9,
          padding='max_length',
          truncation=True,
          return_tensors='pt'
      )
      tokenized_dialogues.append(tokenized_dialogue)

  #padded_input_ids = pad_sequence([dialogue['input_ids'] for dialogue in tokenized_dialogues], batch_first=True)
  input_ids = [dialogue['input_ids'] for dialogue in tokenized_dialogues]
  attention_mask = [dialogue['attention_mask'] for dialogue in tokenized_dialogues]
  return input_ids,attention_mask

#padded_input_ids_train,padded_attention_mask_train = tokenize_padding(train_data['utterances'])
input_ids_train,attention_mask_train = tokenize_padding(train_data['utterances'])
input_ids_val,attention_mask_val = tokenize_padding(val_data['utterances'])
input_ids_test,attention_mask_test = tokenize_padding(test_data['utterances'])

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

100%|██████████| 400/400 [00:00<00:00, 429.79it/s]


In [17]:
from sklearn.metrics import f1_score

def compute_sequence_f1(predictions, labels):
    # predictions and labels should be lists of tensors for each dialogue
    emotion_f1_scores = []
    trigger_f1_scores = []
    for emotion_pred, trigger_pred, emotion_lab, trigger_lab in zip(predictions[0], predictions[1], labels[0], labels[1]):
        emotion_predicted_classes = torch.argmax(emotion_pred, dim=1)
        trigger_predicted_classes = torch.argmax(trigger_pred, dim=1)
        emotion_true_classes = torch.argmax(emotion_lab, dim=1)
        trigger_true_classes = trigger_lab
        emotion_f1 = f1_score(emotion_true_classes.cpu().numpy(), emotion_predicted_classes.cpu().numpy(), average='macro')
        trigger_f1 = f1_score(trigger_true_classes.cpu().numpy(), trigger_predicted_classes.cpu().numpy(), average='macro')
        emotion_f1_scores.append(emotion_f1)
        trigger_f1_scores.append(trigger_f1)
    average_emotion_f1 = torch.tensor(emotion_f1_scores, dtype=torch.float32).mean()
    average_trigger_f1 = torch.tensor(trigger_f1_scores, dtype=torch.float32).mean()
    return average_emotion_f1, average_trigger_f1

def compute_unrolled_sequence_f1(predictions, labels):
    # Flatten all utterances and compute the F1 score
    all_emotion_predicted_classes = torch.argmax(torch.cat(predictions[0], dim=0), dim=1)
    all_trigger_predicted_classes = torch.argmax(torch.cat(predictions[1], dim=0), dim=1)
    all_emotion_true_classes = torch.argmax(torch.cat(labels[0], dim=0), dim=1)
    all_trigger_true_classes = torch.cat(labels[1], dim=0)
    unrolled_emotion_f1 = f1_score(all_emotion_true_classes.cpu().numpy(), all_emotion_predicted_classes.cpu().numpy(), average='macro')
    unrolled_trigger_f1 = f1_score(all_trigger_true_classes.cpu().numpy(), all_trigger_predicted_classes.cpu().numpy(), average='macro')
    unrolled_emotion_f1_tensor = torch.tensor(unrolled_emotion_f1, dtype=torch.float32)
    unrolled_trigger_f1_tensor = torch.tensor(unrolled_trigger_f1, dtype=torch.float32)
    return unrolled_emotion_f1_tensor, unrolled_trigger_f1_tensor

In [18]:
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, emotions, triggers):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.emotions = emotions
        self.triggers = triggers

    def __len__(self):
        return len(self.emotions)

    def __getitem__(self, idx):
        input_ids = self.input_ids[idx]
        attention_mask = self.attention_mask[idx]
        emotion = self.emotions[idx]
        trigger = self.triggers[idx]

        emotion_labels = torch.tensor(emotion, dtype=torch.float32)
        trigger_label = torch.tensor(trigger, dtype=torch.long)
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'emotion_labels': emotion_labels,
            'trigger_label': trigger_label
        }

In [19]:
import torch
from transformers import BertModel


class CustomBERTModel(torch.nn.Module):
    def __init__(self, freeze_embeddings=True, hidden_size=128, num_layers=1, bidirectional=True):
        super(CustomBERTModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        if freeze_embeddings:
            for name, param in self.bert.named_parameters():
                if 'embeddings' in name:
                    param.requires_grad = False

        input_size = self.bert.config.hidden_size
        linear_hidden_size = 64  # Adjust this based on your requirements

        # Linear layer before LSTM
        self.linear_before_lstm = torch.nn.Linear(input_size, linear_hidden_size)

        # LSTM layer
        self.lstm = torch.nn.LSTM(
            input_size=linear_hidden_size,  # Input size after the linear layer
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=bidirectional,
            batch_first=True
        )

        # Linear layer after LSTM
        self.linear_after_lstm = torch.nn.Linear(hidden_size * 2 if bidirectional else hidden_size, linear_hidden_size)

        self.dropout = torch.nn.Dropout(0.5)
        # Linear layer for emotion classification
        self.emotion_head = torch.nn.Linear(linear_hidden_size, len(emotions))

        # Linear layer for trigger classification
        self.trigger_head = torch.nn.Linear(linear_hidden_size, len(triggers))

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        pooled_output = outputs['pooler_output']

        # Linear layer before LSTM
        linear_before_lstm_output = torch.nn.functional.relu(self.linear_before_lstm(pooled_output))

        # LSTM layer
        lstm_input = linear_before_lstm_output.unsqueeze(1).expand(-1, 9, -1)
        lstm_output, _ = self.lstm(lstm_input)
        # Extract the output from the last time step
        lstm_output = lstm_output[:, -1, :]

        # Linear layer after LSTM
        linear_after_lstm_output = torch.nn.functional.relu(self.linear_after_lstm(lstm_output))

        linear_output = self.dropout(linear_after_lstm_output)
        # Emotion head
        emotion_logits = self.emotion_head(linear_output)

        # Trigger head
        trigger_logits = self.trigger_head(linear_output)

        return emotion_logits, trigger_logits

In [20]:
train_dataset = CustomDataset(input_ids_train, attention_mask_train, train_data['emotions'],
                              train_data['triggers'])
validation_dataset = CustomDataset(input_ids_val, attention_mask_val, val_data['emotions'],
                             val_data['triggers'])
test_dataset = CustomDataset(input_ids_test, attention_mask_test, test_data['emotions'],
                             test_data['triggers'])

In [21]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    emotion_labels = [item['emotion_labels'] for item in batch]#torch.stack([item['emotion_labels'] for item in batch], dim=0)
    trigger_label = [item['trigger_label'] for item in batch]#torch.stack([item['trigger_label'] for item in batch], dim=0)

    #input_ids = pad_sequence([torch.stack(item['input_ids']) for item in batch], batch_first=True)
    #attention_mask = pad_sequence([torch.stack(item['attention_mask']) for item in batch], batch_first=True)
    return input_ids,attention_mask,emotion_labels,trigger_label
    #return {'input_ids': input_ids, 'attention_mask': attention_mask, 'emotion_labels': emotion_labels, 'trigger_label': trigger_label

## Bert

In [22]:
criterion_emotion = torch.nn.CrossEntropyLoss() # can tinker with the loss function, change to a different one
criterion_trigger = torch.nn.BCEWithLogitsLoss()

num_epochs = 10
batch_size = 20

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
freezed_embeddings = True
custom_Bert_Model = CustomBERTModel()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
custom_Bert_Model = custom_Bert_Model.to(device)
#optimizer = torch.optim.Adam(custom_Bert_Model.parameters(), lr=5e-5)
optimizer = torch.optim.AdamW(custom_Bert_Model.parameters(), lr=1e-5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
def test_bert(mode='validation'):
# Usage in the eval loop
  sequence_f1_scores_emotion = []
  sequence_f1_scores_trigger = []
  unrolled_predictions_emotion = []
  unrolled_predictions_trigger = []
  unrolled_labels_emotion = []
  unrolled_labels_trigger = []
  sequence_f1_scores = []

  batch_size = 1
  #test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
  if mode == 'validation':
    loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
  elif mode == 'test':
    loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  with torch.no_grad():
      for batch in tqdm(loader, desc='Evaluation', leave=False):
          input_ids = batch['input_ids'].squeeze().to(device)
          attention_mask = batch['attention_mask'].squeeze().to(device)
          emotion_labels = batch['emotion_labels'].squeeze().to(device)
          trigger_label = batch['trigger_label'].squeeze().to(device)

          emotion_logits, trigger_logits = custom_Bert_Model(input_ids, attention_mask)

          # Store predictions and labels for later unrolled F1 computation
          unrolled_predictions_emotion.append(emotion_logits)
          unrolled_labels_emotion.append(emotion_labels)
          unrolled_predictions_trigger.append(trigger_logits)
          unrolled_labels_trigger.append(trigger_label)

          # Convert logits to probabilities and then to class predictions
          #predicted_classes = torch.argmax(emotion_logits, dim=1)
          #true_classes = torch.argmax(emotion_labels, dim=1)

          # Compute F1 for the current sequence (dialogue)
          #sequence_f1 = f1_score(true_classes.cpu().numpy(), predicted_classes.cpu().numpy(), average='macro')
          #sequence_f1_scores.append(sequence_f1)

  # Compute the average Sequence F1 for emotions and triggers
  average_sequence_f1_emotion, average_sequence_f1_trigger = compute_sequence_f1(
      [unrolled_predictions_emotion, unrolled_predictions_trigger],
      [unrolled_labels_emotion, unrolled_labels_trigger]
  )

  # Compute the Unrolled Sequence F1 for emotions and triggers
  unrolled_sequence_f1_emotion, unrolled_sequence_f1_trigger = compute_unrolled_sequence_f1(
      [unrolled_predictions_emotion, unrolled_predictions_trigger],
      [unrolled_labels_emotion, unrolled_labels_trigger]
  )

  # Print the F1 scores for emotions and triggers
  print(f"Average Sequence F1 (Emotion):  {average_sequence_f1_emotion:03f}")
  print(f"Average Sequence F1 (Trigger): {average_sequence_f1_trigger:03f}")
  print(f"Unrolled Sequence F1 (Emotion): {unrolled_sequence_f1_emotion.item():03f}")
  print(f"Unrolled Sequence F1 (Trigger): {unrolled_sequence_f1_trigger.item():03f}")
  return average_sequence_f1_emotion, average_sequence_f1_trigger, unrolled_sequence_f1_emotion, unrolled_sequence_f1_trigger

In [24]:
for epoch in range(num_epochs):
    custom_Bert_Model.train()
    total_loss = 0.0

    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}', leave=False):
        input_ids,attention_mask,emotion_labels,trigger_label = batch

        # Zero the gradients on the optimizer
        optimizer.zero_grad()

        for el in range(batch_size):
          emotion_loss = 0.0
          trigger_loss = 0.0

          input_ids_el = input_ids[el].squeeze().to(device)
          attention_mask_el = attention_mask[el].squeeze().to(device)
          emotion_labels_el = emotion_labels[el].squeeze().to(device)
          trigger_label_el = trigger_label[el].squeeze().to(device)

          # Forward pass
          emotion_logits, trigger_logits = custom_Bert_Model(input_ids_el, attention_mask_el)
          # Compute the loss for both emotion and trigger

          emotion_loss += criterion_emotion(emotion_logits, torch.argmax(emotion_labels_el, dim=1))
          trigger_loss += criterion_trigger(torch.argmax(trigger_logits, dim=1).float(), trigger_label_el.float())

        # Combine losses for backpropagation
        loss = (emotion_loss + trigger_loss)/batch_size

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    # Compute the average loss
    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Loss: {average_loss}")
    test_bert('validation')

Epoch 1, Average Loss: 0.1503024948760867


Average Sequence F1 (Emotion):  0.181846
Average Sequence F1 (Trigger): 0.381783
Unrolled Sequence F1 (Emotion): 0.085187
Unrolled Sequence F1 (Trigger): 0.349896


Epoch 2, Average Loss: 0.14464529165998102


Average Sequence F1 (Emotion):  0.184224
Average Sequence F1 (Trigger): 0.378526
Unrolled Sequence F1 (Emotion): 0.085495
Unrolled Sequence F1 (Trigger): 0.348740


Epoch 3, Average Loss: 0.14120144257321954


...

Average Sequence F1 (Emotion):  0.244442
Average Sequence F1 (Trigger): 0.380716
Unrolled Sequence F1 (Emotion): 0.150634
Unrolled Sequence F1 (Trigger): 0.350082


Epoch 9, Average Loss: 0.13216728786937892


Average Sequence F1 (Emotion):  0.256054
Average Sequence F1 (Trigger): 0.381723
Unrolled Sequence F1 (Emotion): 0.167504
Unrolled Sequence F1 (Trigger): 0.350187


Epoch 10, Average Loss: 0.13169767963699996


Average Sequence F1 (Emotion):  0.256995
Average Sequence F1 (Trigger): 0.383815
Unrolled Sequence F1 (Emotion): 0.166544
Unrolled Sequence F1 (Trigger): 0.351787


In [25]:
test_bert('test')

Average Sequence F1 (Emotion):  0.263634
Average Sequence F1 (Trigger): 0.300903
Unrolled Sequence F1 (Emotion): 0.159998
Unrolled Sequence F1 (Trigger): 0.255204


(tensor(0.2636), tensor(0.3009), tensor(0.1600), tensor(0.2552))

In [26]:
def save_best_model(model, optimizer, epoch, loss, best_loss, path='best_model.pth'):
    """
    Save the best model based on validation loss.
    """
    if loss < best_loss:
        best_loss = loss
        state = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': best_loss
        }
        torch.save(state, path)
    return best_loss

## GRIDSEARCH

In [27]:
import itertools
#best_loss = float('inf')
# Define the grid search parameters
lr_values = [1e-5, 5e-5]
hidden_size_values = [64, 128]
batch_size_values = [20, 32]

# Iterate through all combinations
for lr, hidden_size, batch_size in itertools.product(lr_values, hidden_size_values, batch_size_values):
    # Define your model, optimizer, and other necessary components with the current parameters
    custom_Bert_Model = CustomBERTModel(hidden_size=hidden_size)
    custom_Bert_Model = custom_Bert_Model.to(device)
    optimizer = torch.optim.AdamW(custom_Bert_Model.parameters(), lr=lr)

    # DataLoader with the current batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    num_epochs = 7
    # Training loop and validation code here, using the current lr, hidden_size, and batch_size
    for epoch in range(num_epochs):
      custom_Bert_Model.train()
      total_loss = 0.0

      for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}', leave=False):
          input_ids,attention_mask,emotion_labels,trigger_label = batch

          # Zero the gradients on the optimizer
          optimizer.zero_grad()

          for el in range(batch_size):
            emotion_loss = 0.0
            trigger_loss = 0.0

            input_ids_el = input_ids[el].squeeze().to(device)
            attention_mask_el = attention_mask[el].squeeze().to(device)
            emotion_labels_el = emotion_labels[el].squeeze().to(device)
            trigger_label_el = trigger_label[el].squeeze().to(device)

            # Forward pass
            emotion_logits, trigger_logits = custom_Bert_Model(input_ids_el, attention_mask_el)
            # Compute the loss for both emotion and trigger

            emotion_loss += criterion_emotion(emotion_logits, torch.argmax(emotion_labels_el, dim=1))
            trigger_loss += criterion_trigger(torch.argmax(trigger_logits, dim=1).float(), trigger_label_el.float())

          # Combine losses for backpropagation
          loss = (emotion_loss + trigger_loss)/batch_size

          loss.backward()
          optimizer.step()

          total_loss += loss.item()
      # Compute the average loss
      average_loss = total_loss / len(train_loader)
      print(f"Epoch {epoch + 1}, Average Loss: {average_loss}")
      validation_loss = test_bert('validation')
      #best_loss = save_best_model(custom_Bert_Model, optimizer, epoch, validation_loss, best_loss)

    # Print or log the results for each combination
    print(f"LR: {lr}, Hidden Size: {hidden_size}, Batch Size: {batch_size}")#, Best Validation Loss: {best_loss}")
    test_loss = test_bert('test')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1, Average Loss: 0.15261719641275703


Average Sequence F1 (Emotion):  0.138146
Average Sequence F1 (Trigger): 0.382165
Unrolled Sequence F1 (Emotion): 0.121567
Unrolled Sequence F1 (Trigger): 0.350196


Epoch 2, Average Loss: 0.15134222977794706


Average Sequence F1 (Emotion):  0.211842
Average Sequence F1 (Trigger): 0.380466
Unrolled Sequence F1 (Emotion): 0.167281
Unrolled Sequence F1 (Trigger): 0.349375


...

Epoch 6, Average Loss: 0.0773740902170539


Average Sequence F1 (Emotion):  0.182305
Average Sequence F1 (Trigger): 0.181875
Unrolled Sequence F1 (Emotion): 0.085639
Unrolled Sequence F1 (Trigger): 0.218069


Epoch 7, Average Loss: 0.07652293354272842


Average Sequence F1 (Emotion):  0.183415
Average Sequence F1 (Trigger): 0.179714
Unrolled Sequence F1 (Emotion): 0.089072
Unrolled Sequence F1 (Trigger): 0.219688
LR: 5e-05, Hidden Size: 128, Batch Size: 32


Average Sequence F1 (Emotion):  0.186774
Average Sequence F1 (Trigger): 0.189988
Unrolled Sequence F1 (Emotion): 0.095758
Unrolled Sequence F1 (Trigger): 0.208914


## Majority classifier

In [39]:
# Reinitialise train, val and test loaders without collate_fn
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [40]:
import torch

def find_majority_class(train_loader):
    # Initialize counters
    emotion_counts = torch.zeros(7) # Assuming there are 7 unique emotions
    trigger_counts = torch.zeros(2) # There are 2 classes for triggers: present or not
    negative_trigger_counts = 0
    positive_trigger_counts = 0
    # Iterate over the training dataset to count the labels
    for batch in train_loader:
        emotion_labels = batch['emotion_labels'].squeeze()
        trigger_labels = batch['trigger_label'].squeeze()
        #print(trigger_labels,torch.sum(trigger_labels, dim=0),(trigger_labels == 0).sum())
        # Sum up the counts for each class
        positive_trigger_counts += torch.sum(trigger_labels, dim=0)
        # Count the zeros for the negative class (absence of a trigger)
        # Since one-hot encoding, the absence is just the inverse of the presence
        negative_trigger_counts += torch.sum(1 - trigger_labels, dim=0)
        emotion_counts += torch.sum(emotion_labels, dim=0)

    trigger_counts[0] = negative_trigger_counts
    trigger_counts[1] = positive_trigger_counts
    print(trigger_counts)
    print(emotion_counts)
    # Find the index with the maximum count for emotions and triggers
    majority_emotion = torch.zeros_like(emotion_counts)
    majority_emotion[torch.argmax(emotion_counts)] = 1
    majority_trigger = torch.zeros_like(trigger_counts)
    majority_trigger[torch.argmax(trigger_counts)] = 1

    return majority_emotion, majority_trigger

# Let's assume that 'train_loader' is a DataLoader for your training dataset
# You need to replace 'train_loader' with the actual DataLoader for your dataset
majority_emotion, majority_trigger = find_majority_class(train_loader)
majority_emotion, majority_trigger

tensor([23524.,  4289.])
tensor([ 3025.,   816.,   917.,  5123., 12228.,  1929.,  3775.])


(tensor([0., 0., 0., 0., 1., 0., 0.]), tensor([1., 0.]))

In [42]:
from sklearn.metrics import f1_score
from tqdm.auto import tqdm

def majority_classifier(majority_emotion, majority_trigger, test_loader):
    all_emotion_predictions = []
    all_trigger_predictions = []
    all_emotion_labels = []
    all_trigger_labels = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc='Evaluation', leave=False):
            emotion_labels = batch['emotion_labels']
            trigger_labels = batch['trigger_label']

            for emotion_lab, trigger_lab in zip(emotion_labels, trigger_labels):
                # Ensure we have at least 1 dimension
                if emotion_lab.ndim == 1 and emotion_lab.size(0) == 1:
                    emotion_lab = emotion_lab.unsqueeze(0)
                if trigger_lab.ndim == 1 and trigger_lab.size(0) == 1:
                    trigger_lab = trigger_lab.unsqueeze(0)

                # Store the labels
                all_emotion_labels.append(emotion_lab)
                all_trigger_labels.append(trigger_lab)

                # Repeat the majority class prediction to match the number of utterances
                emotion_predictions = majority_emotion.repeat(emotion_lab.size(0), 1)
                #print(emotion_predictions)

                trigger_predictions = majority_trigger.repeat(trigger_lab.size(0), 1)

                # Store the predictions
                all_emotion_predictions.append(emotion_predictions)
                all_trigger_predictions.append(trigger_predictions)

    # Use the stored predictions and labels to calculate sequence F1 and unrolled sequence F1
    average_sequence_f1_emotion, average_sequence_f1_trigger = compute_sequence_f1(
        [all_emotion_predictions, all_trigger_predictions],
        [all_emotion_labels, all_trigger_labels]
    )

    unrolled_sequence_f1_emotion, unrolled_sequence_f1_trigger = compute_unrolled_sequence_f1(
        [all_emotion_predictions, all_trigger_predictions],
        [all_emotion_labels, all_trigger_labels]
    )

    return average_sequence_f1_emotion, average_sequence_f1_trigger, unrolled_sequence_f1_emotion, unrolled_sequence_f1_trigger

# Assume majority_emotion and majority_trigger are tensors of the majority class (one-hot encoded)
# and test_loader is your DataLoader instance for the test dataset.
average_f1_emotion, average_f1_trigger, unrolled_f1_emotion, unrolled_f1_trigger = majority_classifier(majority_emotion, majority_trigger, test_loader)

print(f"Average Sequence F1 (Emotion): {average_f1_emotion}")
print(f"Average Sequence F1 (Trigger): {average_f1_trigger}")
print(f"Unrolled Sequence F1 (Emotion): {unrolled_f1_emotion}")
print(f"Unrolled Sequence F1 (Trigger): {unrolled_f1_trigger}")

Evaluation:   0%|          | 0/400 [00:00<?, ?it/s]

Average Sequence F1 (Emotion): 0.17765390872955322
Average Sequence F1 (Trigger): 0.0
Unrolled Sequence F1 (Emotion): 0.08463548868894577
Unrolled Sequence F1 (Trigger): 0.0


## Random

In [43]:
import torch

def random_classifier(test_loader, emotion_distribution, trigger_distribution):
    all_emotion_predictions = []
    all_trigger_predictions = []
    all_emotion_labels = []
    all_trigger_labels = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc='Evaluation', leave=False):
            emotion_labels = batch['emotion_labels']
            trigger_labels = batch['trigger_label']

            for emotion_lab, trigger_lab in zip(emotion_labels, trigger_labels):
                # Ensure we have at least 1 dimension
                if emotion_lab.ndim == 1 and emotion_lab.size(0) == 1:
                    emotion_lab = emotion_lab.unsqueeze(0)
                if trigger_lab.ndim == 1 and trigger_lab.size(0) == 1:
                    trigger_lab = trigger_lab.unsqueeze(0)

                # Store the labels
                all_emotion_labels.append(emotion_lab)
                all_trigger_labels.append(trigger_lab)

                # Generate random predictions for emotions
                random_emotion_predictions = torch.randint(0, 2, (emotion_lab.size(0), 7))  # Randomly 0 or 1 for each emotion
                all_emotion_predictions.append(random_emotion_predictions.float())

                # Generate random predictions for triggers based on the training distribution
                random_trigger_probs = torch.rand((trigger_lab.size(0), 1))
                random_trigger_predictions = (random_trigger_probs < trigger_distribution).long()  # Binary prediction based on distribution
                random_trigger_predictions = torch.cat((random_trigger_predictions, 1 - random_trigger_predictions), dim=1)  # Make it one-hot
                all_trigger_predictions.append(random_trigger_predictions.float())

    # Calculate the F1 scores using your metric functions
    average_sequence_f1_emotion, average_sequence_f1_trigger = compute_sequence_f1(
        [all_emotion_predictions, all_trigger_predictions],
        [all_emotion_labels, all_trigger_labels]
    )

    unrolled_sequence_f1_emotion, unrolled_sequence_f1_trigger = compute_unrolled_sequence_f1(
        [all_emotion_predictions, all_trigger_predictions],
        [all_emotion_labels, all_trigger_labels]
    )

    return average_sequence_f1_emotion, average_sequence_f1_trigger, unrolled_sequence_f1_emotion, unrolled_sequence_f1_trigger

# You need to provide the distribution for the trigger class from your training data
# For example, if 30% of your training samples have a trigger, trigger_distribution should be 0.3
trigger_distribution = 0.5  # Replace with your actual distribution

# Now call your random classifier function
average_f1_emotion, average_f1_trigger, unrolled_f1_emotion, unrolled_f1_trigger = random_classifier(
    test_loader,
    emotion_distribution=None,  # Not used currently as we're assuming a uniform distribution
    trigger_distribution=trigger_distribution
)

print(f"Random Classifier Average Sequence F1 (Emotion): {average_f1_emotion}")
print(f"Random Classifier Average Sequence F1 (Trigger): {average_f1_trigger}")
print(f"Random Classifier Unrolled Sequence F1 (Emotion): {unrolled_f1_emotion}")
print(f"Random Classifier Unrolled Sequence F1 (Trigger): {unrolled_f1_trigger}")

Evaluation:   0%|          | 0/400 [00:00<?, ?it/s]

Random Classifier Average Sequence F1 (Emotion): 0.05748957768082619
Random Classifier Average Sequence F1 (Trigger): 0.21685552597045898
Random Classifier Unrolled Sequence F1 (Emotion): 0.07542110979557037
Random Classifier Unrolled Sequence F1 (Trigger): 0.20085106790065765


## Bert with context

In [80]:
import torch
from transformers import BertModel
import torch.nn as nn

class BERT_LSTM_5(torch.nn.Module):
    def __init__(self, num_emotions, num_triggers, freeze_embeddings=True):
        super(BERT_LSTM_5, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        if freeze_embeddings:
            for name, param in self.bert.named_parameters():
                if 'embeddings' in name:
                    param.requires_grad = False

        self.lstm_hidden_size = self.bert.config.hidden_size

        self.lstm = nn.LSTM(input_size=self.lstm_hidden_size,
                            hidden_size=self.lstm_hidden_size,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=True)

        # Dropout for L1 Regularization
        self.dropout = nn.Dropout(p=0.2)

        # Instead of applying custom attention, we'll use the output from LSTM directly for classification
        self.emotion_head = nn.Linear(self.lstm_hidden_size * 2, num_emotions)  # *2 for bidirectional
        self.trigger_head = nn.Linear(self.lstm_hidden_size * 2, num_triggers)

    def forward(self, input_ids, attention_mask):
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        bert_sequence_output = bert_outputs.last_hidden_state

        lstm_output, (h_n, c_n) = self.lstm(bert_sequence_output)

        # Instead of applying attention, use the last hidden states directly
        # Concatenate the final forward and backward hidden states
        h_n = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)

        # Apply dropout to the concatenated LSTM outputs
        h_n = self.dropout(h_n)

        emotion_logits = self.emotion_head(h_n)
        trigger_logits = self.trigger_head(h_n)

        return emotion_logits, trigger_logits


In [81]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

num_emotions = 7
num_triggers = 2

# Instantiate the model
BERT_lstm = BERT_LSTM_5(num_emotions,num_triggers).to(device)
#optimizer = AdamW(filter(lambda p: p.requires_grad, custom_Bert_Model.parameters()), lr=5e-5)
# optimizer = torch.optim.SGD(BERT_lstm.parameters(), lr=1e-5)
optimizer = torch.optim.AdamW(BERT_lstm.parameters(), lr=1e-5, weight_decay=1e-4)
# optimizer = torch.optim.AdamW(BERT_lstm.parameters(), lr=1e-3,weight_decay = 5e-4)
criterion = torch.nn.CrossEntropyLoss()
# Define the learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, verbose=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [82]:
num_epochs = 8
batch_size = 1
for epoch in range(num_epochs):
    BERT_lstm.train()
    total_loss = 0.0
    loss_emotion = 0.0
    loss_trigger = 0.0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}', leave=False):
        input_ids = batch['input_ids'].squeeze().to(device)
        attention_mask = batch['attention_mask'].squeeze().to(device)
        emotion_labels = batch['emotion_labels'].squeeze().to(device)
        trigger_label = batch['trigger_label'].squeeze().to(device)

        # Zero the gradients on the optimizer
        optimizer.zero_grad()

        # Forward pass
        emotion_logits, trigger_logits = BERT_lstm(input_ids, attention_mask)

        # Compute the loss for both emotion and trigger
        emotion_loss = criterion(emotion_logits, torch.argmax(emotion_labels, dim=1))
        trigger_loss = criterion(trigger_logits, trigger_label)

        # Combine losses for backpropagation
        loss = emotion_loss + trigger_loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loss_emotion += emotion_loss
        loss_trigger += trigger_loss

    # Step the scheduler after each epoch
    scheduler.step(total_loss)

    # Compute the average loss
    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Loss: {average_loss}")
    print(f"Epoch {epoch + 1}, Emotion Loss: {loss_emotion/len(train_loader)}")
    print(f"Epoch {epoch + 1}, Trigger Loss: {loss_trigger/len(train_loader)}")

Epoch 1:   0%|          | 0/3200 [00:00<?, ?it/s]

Epoch 1, Average Loss: 1.5645286426180973
Epoch 1, Emotion Loss: 1.074985146522522
Epoch 1, Trigger Loss: 0.48954230546951294


Epoch 2:   0%|          | 0/3200 [00:00<?, ?it/s]

Epoch 2, Average Loss: 0.9662123980163596
Epoch 2, Emotion Loss: 0.486752986907959
Epoch 2, Trigger Loss: 0.4794595539569855


Epoch 3:   0%|          | 0/3200 [00:00<?, ?it/s]

...

Epoch 6, Average Loss: 0.5683110034570563
Epoch 6, Emotion Loss: 0.13789403438568115
Epoch 6, Trigger Loss: 0.4304167330265045


Epoch 7:   0%|          | 0/3200 [00:00<?, ?it/s]

Epoch 7, Average Loss: 0.5380457119905623
Epoch 7, Emotion Loss: 0.11929257214069366
Epoch 7, Trigger Loss: 0.4187520742416382


Epoch 8:   0%|          | 0/3200 [00:00<?, ?it/s]

Epoch 8, Average Loss: 0.5229891760356259
Epoch 8, Emotion Loss: 0.11352823674678802
Epoch 8, Trigger Loss: 0.40946078300476074


In [83]:
# Usage in the eval loop
sequence_f1_scores_emotion = []
sequence_f1_scores_trigger = []
unrolled_predictions_emotion = []
unrolled_predictions_trigger = []
unrolled_labels_emotion = []
unrolled_labels_trigger = []
sequence_f1_scores = []



with torch.no_grad():
    for batch in tqdm(test_loader, desc='Evaluation', leave=False):
        input_ids = batch['input_ids'].squeeze().to(device)
        attention_mask = batch['attention_mask'].squeeze().to(device)
        emotion_labels = batch['emotion_labels'].squeeze().to(device)
        trigger_label = batch['trigger_label'].squeeze().to(device)

        emotion_logits, trigger_logits = BERT_lstm(input_ids, attention_mask)

        # Store predictions and labels for later unrolled F1 computation
        unrolled_predictions_emotion.append(emotion_logits)
        unrolled_labels_emotion.append(emotion_labels)
        unrolled_predictions_trigger.append(trigger_logits)
        unrolled_labels_trigger.append(trigger_label)

        # Convert logits to probabilities and then to class predictions
        predicted_classes = torch.argmax(emotion_logits, dim=1)
        true_classes = torch.argmax(emotion_labels, dim=1)

        # Compute F1 for the current sequence (dialogue)
        sequence_f1 = f1_score(true_classes.cpu().numpy(), predicted_classes.cpu().numpy(), average='macro')
        sequence_f1_scores.append(sequence_f1)

# Compute the average Sequence F1 for emotions and triggers
average_sequence_f1_emotion, average_sequence_f1_trigger = compute_sequence_f1(
    [unrolled_predictions_emotion, unrolled_predictions_trigger],
    [unrolled_labels_emotion, unrolled_labels_trigger]
)

# Compute the Unrolled Sequence F1 for emotions and triggers
unrolled_sequence_f1_emotion, unrolled_sequence_f1_trigger = compute_unrolled_sequence_f1(
    [unrolled_predictions_emotion, unrolled_predictions_trigger],
    [unrolled_labels_emotion, unrolled_labels_trigger]
)

# Print the F1 scores for emotions and triggers
print(f"Average Sequence F1 (Emotion): {average_sequence_f1_emotion:.4f}")
print(f"Average Sequence F1 (Trigger): {average_sequence_f1_trigger:.4f}")
print(f"Unrolled Sequence F1 (Emotion): {unrolled_sequence_f1_emotion.item():.4f}")
print(f"Unrolled Sequence F1 (Trigger): {unrolled_sequence_f1_trigger.item():.4f}")

Evaluation:   0%|          | 0/400 [00:00<?, ?it/s]

Average Sequence F1 (Emotion): 0.3493
Average Sequence F1 (Trigger): 0.0143
Unrolled Sequence F1 (Emotion): 0.3393
Unrolled Sequence F1 (Trigger): 0.0275
